# Лабораторная №3. Обработка последовательностей

#### Imports

In [3]:
import numpy as np
import re
import random
import keras
from keras.layers import Dense, LSTM

### Text preprocessing

In [4]:
text = open("text.txt","r").read()
ad = """************************************
Данная глава взята из открытого источника. Файл был скачан с сайта: https://loghorizont.ru/ 
Если вам понравилась глава: 
 Оставьте комментарий: https://loghorizont.ru/Boevoj-kontinent/
 Отблагодаритьте нашу команду: https://loghorizont.ru/podderzhat-proekt/
 Помочь проекту, кликнув на рекламу: https://loghorizont.ru/
************************************"""
text = text.replace(ad,'')
text = re.sub("Часть", " ", text)
text = re.sub("Глава", " ", text)
text = text.lower()
text = text.replace("ё","е")
text = text.replace("c","с")
text = text.replace("o","о")
text = re.sub(r'[!?;…]', '.', text)
text = re.sub(r'[^а-я\s.]', ' ', text)
text = re.sub(r'\s+', ' ', text)
text = re.sub(r"\.+", ".", text)
text = re.sub(r"\.( \.)+", ".", text)

In [5]:
print(text)

пролог континент ба шу на протяжении веков он был известен как земля изобилия где самым известным кланом который никто не мог превзойти был клан тан. местоположение клана тан было окутано ореолом тайны. большинство людей знало только то что это место находилось примерно на полпути к вершине горы и что в горах где располагался клан тан имелось место с ужасающим названием пик ада. если с вершины пика ада бросить камень в пропасть то можно будет досчитать до прежде чем вы услышите эхо ударившегося о дно камня. именно из за этих секунд превосходящих по количеству даже кругов ада пик ада и получил свое название. молодой человек в серой одежде стоял на вершине пика ада и порывистый ледяной горный ветер был не в состоянии заставить его тело задрожать. на его груди был виден большой знак тан . он состоял в этом клане и серые одежды означали что он был учеником побочной ветви клана тан. в этом году ему исполнилось двадцать девять лет. а поскольку он вступил в клан тан вскоре после своего рожден

In [77]:
def length_filter(min_length, max_length):
    return lambda s: len(s) > min_length and len(s) < max_length
sentances = list(map(lambda s: s.strip() + '.', filter(length_filter(14,120), text.split('.'))))

In [78]:
alphabet = sorted(set(text))
int_to_char = dict(enumerate(alphabet))
char_to_int = {v: k for k, v in int_to_char.items()}


In [79]:
query_length = 12
dataX, dataY = [], []
for sentence in sentances:
    for i in range(len(sentence) - query_length):
        dataX.append(sentence[i:i + query_length])
        dataY.append(sentence[i + query_length])

#### One-hot encoding

In [80]:
X = np.zeros((len(dataX), query_length, len(alphabet)))
Y = np.zeros((len(dataX), len(alphabet)))
for i, sentence in enumerate(dataX):
    for j, char in enumerate(sentence):
        X[i, j, char_to_int[char]] = 1
    Y[i, char_to_int[dataY[i]]] = 1

print(X.shape)
print(Y.shape)

(320010, 12, 34)
(320010, 34)


### LSTM

In [82]:
model = keras.Sequential([
    LSTM(256, input_shape=(X.shape[1], X.shape[2])),
    Dense(Y.shape[1], activation='softmax')
])
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(X, Y, epochs=10)
model.summary()

Epoch 1/10
10001/10001 [==============================] - 150s 15ms/step - loss: 2.0806
Epoch 2/10
10001/10001 [==============================] - 208s 21ms/step - loss: 1.6084
Epoch 3/10
10001/10001 [==============================] - 123s 12ms/step - loss: 1.4468
Epoch 4/10
10001/10001 [==============================] - 106s 11ms/step - loss: 1.3530
Epoch 5/10
10001/10001 [==============================] - 148s 15ms/step - loss: 1.2852
Epoch 6/10
10001/10001 [==============================] - 153s 15ms/step - loss: 1.2302
Epoch 7/10
10001/10001 [==============================] - 100s 10ms/step - loss: 1.1854
Epoch 8/10
10001/10001 [==============================] - 110s 11ms/step - loss: 1.1456
Epoch 9/10
10001/10001 [==============================] - 112s 11ms/step - loss: 1.1112
Epoch 10/10
10001/10001 [==============================] - 110s 11ms/step - loss: 1.0798
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Ou

In [149]:
def predict(query):
    result = query 
    pattern = query[-query_length:]
    for j in range(100):
        x = np.zeros((1, query_length, len(alphabet)))
        for k, char in enumerate(pattern):
            x[0, k, char_to_int[char]] = 1
        next = int_to_char[np.argmax(model.predict(x,  verbose=0))]
        pattern = pattern + next
        pattern = pattern[-query_length:]
        result += next
        if next == '.':
            break
    return result

for i in range(5):
    query = dataX[np.random.randint(len(sentances) - 1)]
    print(f"Query: {query}")
    print(f"Prediction: {predict(query)}")


Query:  ничего из к
Prediction:  ничего из кусок железа он не мог семьдестой конечного нелавя она спокойно сказала на нее не все еще не могла сп
Query: е оружие в м
Prediction: е оружие в месяц и спокойно сказала на нее не все еще не могла способность второго духовного кольца стала возмож
Query: няли никто и
Prediction: няли никто инструментов по себе человека на силу тан саня с сяо ву и не стала полностью выражение его силы были 
Query: кт его сущес
Prediction: кт его существование.
Query: о из них уже
Prediction: о из них уже не стал слишком много волосо в воздухе он сказал в сторону тан саня с сяо ву и не стала полностью в


In [98]:
query = "культивирова"
print(predict(query))

культивирование духовного зверя настолько выражение лица сказала что ты сказал на месте на деле он показал что т


In [99]:
query = "культ манца"
print(predict(query))

культ манца и посмотрел на тан саня в сторону тан саня с сяо ву и не стала полностью выражение его силы были по


### Markov chain

In [121]:
n = 5
markov_chain = dict()
for i in range(len(text) - n - 1):
    l = text[i: i + n]
    r = text[i + n]
    if l not in markov_chain:
        markov_chain[l] = dict()
    if r not in markov_chain[l]:
        markov_chain[l][r] = 1
    else:
        markov_chain[l][r] += 1

In [123]:
for key, value in markov_chain.items():
    count = sum(value.values())
    for k, v in value.items():
        markov_chain[key][k] = v / count

In [130]:
MAX_LEN = 100
def predict_markov_chain(query):
    result = query
    while len(result) < MAX_LEN:
        query = result[-n:]
        if query not in markov_chain:
            return result + '.'
        probs = markov_chain[query]
        next_ind = np.random.choice(np.arange(len(probs)), p=list(probs.values()))
        next = list(probs.keys())[next_ind]
        result += next
        if next == '.':
            break
    return result

In [142]:
for i in range(5):
    start_index = random.randint(0, len(text) - n - 1)
    query = text[start_index:start_index + n]
    print(f"Query: {query}")
    print(f"Prediction: {predict_markov_chain(query)}")

Query: двум 
Prediction: двум уважаемые громко все пять меня спервая пластинчатый белые редисловий для уничто этого же всюду 
Query:  духо
Prediction:  духовного оружие быстрелы создания меня .
Query:  крис
Prediction:  кристальных правду может в глаза плечам.
Query: о все
Prediction: о всегда в тот удар.
Query: улся 
Prediction: улся поступать культивация фиолетового духовном кузнечно элегантную клятву в то же временно за ним.


In [133]:
predict_markov_chain("культиви")

'культивировала и не говорчал четырех метрах и злости.'